# Satellite Location Demo

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed, both in the past and in the future, in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics.

In [83]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Purples_9, Oranges_9
from palettable.colorbrewer.diverging import BrBG_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [2]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

sentinel_2b = load.tle_file(sentinel_2b_url, reload=True, filename="sentinel_2b")

[#################################] 100% sentinel_2b


In [3]:
satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-07-11 16:23:36 UTC>

---

## Calculating the position of a satellite

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +- 1 day from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run).

In [4]:
# Calculate the current UTC time (without microseconds), then creating a time range +/- 24h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides)
eph = load('de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [5]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [6]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords, daytime

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [7]:
while timer <= t2:
    timer, longitude, latitude, coords, daytime = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=15) # ~13 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [8]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
0,SENTINEL-2B,2024-07-12 13:42:56+00:00,POINT (-75.38464 -71.83547),-75.384642,-71.835472,True,2024-07-12 13:42:56
1,SENTINEL-2B,2024-07-12 13:43:11+00:00,POINT (-76.85869 -72.61103),-76.858688,-72.611032,True,2024-07-12 13:43:11
2,SENTINEL-2B,2024-07-12 13:43:26+00:00,POINT (-78.45917 -73.37570),-78.459166,-73.375703,True,2024-07-12 13:43:26
3,SENTINEL-2B,2024-07-12 13:43:41+00:00,POINT (-80.20217 -74.12793),-80.202166,-74.127929,True,2024-07-12 13:43:41
4,SENTINEL-2B,2024-07-12 13:43:56+00:00,POINT (-82.10606 -74.86587),-82.106064,-74.865873,True,2024-07-12 13:43:56
...,...,...,...,...,...,...,...
11516,SENTINEL-2B,2024-07-14 13:41:56+00:00,POINT (-20.90928 71.75638),-20.909281,71.756383,True,2024-07-14 13:41:56
11517,SENTINEL-2B,2024-07-14 13:42:11+00:00,POINT (-22.26571 70.96687),-22.265713,70.966871,True,2024-07-14 13:42:11
11518,SENTINEL-2B,2024-07-14 13:42:26+00:00,POINT (-23.52235 70.16897),-23.522354,70.168968,True,2024-07-14 13:42:26
11519,SENTINEL-2B,2024-07-14 13:42:41+00:00,POINT (-24.68991 69.36363),-24.689911,69.363635,True,2024-07-14 13:42:41


Let's refine our area of interest to see when/where satellites pass overhead. We can use a large bounding box of South Asia for demonstration purposes.

In [9]:
# Florida

min_lon = -89
max_lon = -74
min_lat = 22
max_lat = 32

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

[-89, -74, 22, 32]

In [10]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()
aoi

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
687,SENTINEL-2B,2024-07-12 16:34:41+00:00,POINT (-85.79488 31.86746),-85.794882,31.867457,True,2024-07-12 16:34:41
688,SENTINEL-2B,2024-07-12 16:34:56+00:00,POINT (-86.04003 30.98383),-86.040033,30.983833,True,2024-07-12 16:34:56
689,SENTINEL-2B,2024-07-12 16:35:11+00:00,POINT (-86.28186 30.09984),-86.281856,30.099836,True,2024-07-12 16:35:11
690,SENTINEL-2B,2024-07-12 16:35:26+00:00,POINT (-86.52052 29.21548),-86.520522,29.215481,True,2024-07-12 16:35:26
691,SENTINEL-2B,2024-07-12 16:35:41+00:00,POINT (-86.75620 28.33078),-86.756195,28.330782,True,2024-07-12 16:35:41
692,SENTINEL-2B,2024-07-12 16:35:56+00:00,POINT (-86.98903 27.44576),-86.989030,27.445756,True,2024-07-12 16:35:56
693,SENTINEL-2B,2024-07-12 16:36:11+00:00,POINT (-87.21917 26.56042),-87.219173,26.560416,True,2024-07-12 16:36:11
694,SENTINEL-2B,2024-07-12 16:36:26+00:00,POINT (-87.44677 25.67478),-87.446766,25.674775,True,2024-07-12 16:36:26
695,SENTINEL-2B,2024-07-12 16:36:41+00:00,POINT (-87.67194 24.78885),-87.671941,24.788848,True,2024-07-12 16:36:41
696,SENTINEL-2B,2024-07-12 16:36:56+00:00,POINT (-87.89483 23.90265),-87.894827,23.902648,True,2024-07-12 16:36:56


Next, we can visualize these points on a map. They are color coded using a continuous color scale, with white being `now` and darker purple being further after `now`.

In [11]:
# This creates a range from 0-1 to define our colormap.
time_norm = (aoi.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, Purples_9)

In [12]:
layer = ScatterplotLayer.from_geopandas(
    aoi,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame.

---

## Calculating the position of a satellite constellation

Usually, humanitarians aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [13]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

planet_swarm = load.tle_file(planet_url, reload=True, filename="planet_swarm")
planet_swarm[0]

[#################################] 100% planet_swarm


<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-07-11 13:15:24 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [14]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords, daytime = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
        rows.append(row)    

    timer += timedelta(minutes=3) # ~ 10 min

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [15]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [16]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
aoi_planet_day[aoi_planet_day["timestamp"] > now]

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
1553,SKYSAT-C12,2024-07-12 14:15:56+00:00,POINT (-79.15461 28.89216),-79.154609,28.892164,True,2024-07-12 14:15:56
2946,SKYSAT-C3,2024-07-12 14:45:56+00:00,POINT (-80.78268 30.48031),-80.782677,30.480312,True,2024-07-12 14:45:56
2964,FLOCK 4S-11,2024-07-12 14:45:56+00:00,POINT (-81.97362 26.98495),-81.973615,26.984953,True,2024-07-12 14:45:56
3097,SKYSAT-C15,2024-07-12 14:48:56+00:00,POINT (-85.82170 30.53369),-85.821701,30.533691,True,2024-07-12 14:48:56
3586,FLOCK 4Y-16,2024-07-12 14:57:56+00:00,POINT (-74.22264 30.34258),-74.222645,30.342576,True,2024-07-12 14:57:56
...,...,...,...,...,...,...,...
76073,FLOCK 4X-29,2024-07-13 16:51:56+00:00,POINT (-88.75237 24.99195),-88.752373,24.991947,True,2024-07-13 16:51:56
84428,SKYSAT-C10,2024-07-13 19:51:56+00:00,POINT (-85.37147 23.18044),-85.371467,23.180436,True,2024-07-13 19:51:56
84990,SKYSAT-C8,2024-07-13 20:03:56+00:00,POINT (-88.74132 24.06610),-88.741324,24.066098,True,2024-07-13 20:03:56
100953,SKYSAT-C12,2024-07-14 01:45:56+00:00,POINT (-78.75536 24.31752),-78.755362,24.317517,True,2024-07-14 01:45:56


In [17]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet = (aoi_planet_day.timestamp - t1) / (t2 - t1)

colors_planet = apply_continuous_cmap(time_norm_planet, Purples_9)

Let's visualize this with the same color scheme we used before.

In [18]:
layer2 = ScatterplotLayer.from_geopandas(
    aoi_planet_day,
    # extensions=[filter_extension],
    get_fill_color=colors_planet,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m2 = Map(
    layer2,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

In reality, satellites are taking more images than the amount of time steps that we arbitrarily set. A line would better represent the path these satellites take. The area of capture of satellites also varies, which a point doesn't represent.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.

---

In [19]:
# OLD

paths = planet_swarm_gdf.groupby('satellite')['coordinates'].apply(lambda x: LineString(x.tolist()))
paths = gpd.GeoDataFrame(paths, geometry='coordinates')
paths = paths.reset_index()
paths

,satellite,coordinates
0,FLOCK 4Q-1,"LINESTRING (125.05396 44.07497, 120.73349 55.2..."
1,FLOCK 4Q-10,"LINESTRING (-56.27554 -49.36140, -61.55296 -60..."
2,FLOCK 4Q-11,"LINESTRING (144.56009 -58.64681, 139.66628 -47..."
3,FLOCK 4Q-12,"LINESTRING (125.92823 40.49756, 122.07290 51.7..."
4,FLOCK 4Q-13,"LINESTRING (128.77338 25.45880, 125.99637 36.7..."
...,...,...
135,SKYSAT-C5,"LINESTRING (92.31192 70.01893, 67.45275 79.845..."
136,SKYSAT-C6,"LINESTRING (-1.23006 -38.68379, -4.87435 -49.9..."
137,SKYSAT-C7,"LINESTRING (-67.21479 -82.24344, -137.70823 -7..."
138,SKYSAT-C8,"LINESTRING (177.68926 36.76988, 174.21737 48.0..."


In [51]:
def create_individual_linestrings(coords, timestamps):
    coords_list = coords.tolist()
    timestamps_list = timestamps.tolist()
    line_segments = [
        {'linestring': LineString([coords_list[i], coords_list[i + 1]]), 'timestamp': timestamps_list[i]} 
        for i in range(len(coords_list) - 1)
    ]
    return line_segments

In [53]:
new_rows = []

for satellite, group in aoi_planet.groupby('satellite'):
    line_segments = create_individual_linestrings(group['coordinates'], group['timestamp'])
    for segment in line_segments:
        new_rows.append({'satellite': satellite, 'linestring': segment['linestring'], 'timestamp': segment['timestamp']})

path_segments = gpd.GeoDataFrame(new_rows, geometry='linestring')
path_segments

,satellite,linestring,timestamp
0,FLOCK 4Q-1,"LINESTRING (-74.48719 27.99074, -84.11069 29.1...",2024-07-12 15:45:56+00:00
1,FLOCK 4Q-1,"LINESTRING (-84.11069 29.11455, -79.85466 30.5...",2024-07-13 03:51:56+00:00
2,FLOCK 4Q-10,"LINESTRING (-84.91516 31.85343, -80.93383 28.8...",2024-07-12 16:30:56+00:00
3,FLOCK 4Q-11,"LINESTRING (-81.07333 29.38098, -76.67739 28.7...",2024-07-12 16:12:56+00:00
4,FLOCK 4Q-11,"LINESTRING (-76.67739 28.75495, -86.23820 27.9...",2024-07-13 15:54:56+00:00
...,...,...,...
176,SKYSAT-C4,"LINESTRING (-74.48291 24.05652, -84.97990 22.4...",2024-07-13 01:24:56+00:00
177,SKYSAT-C5,"LINESTRING (-77.15038 30.91307, -84.85299 27.7...",2024-07-13 02:03:56+00:00
178,SKYSAT-C7,"LINESTRING (-80.31838 22.45945, -87.39486 28.6...",2024-07-12 19:30:56+00:00
179,SKYSAT-C7,"LINESTRING (-87.39486 28.64673, -80.62626 28.5...",2024-07-13 07:30:56+00:00


In [54]:
viz(paths[paths["satellite"] == "SKYSAT-C16"])

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [92]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet_swarm = (path_segments.timestamp - t1) / (t2 - t1)

colors_planet_swarm = apply_continuous_cmap(time_norm_planet_swarm, Oranges_9, alpha=1)
colors_planet_swarm
len(colors_planet_swarm)

181

In [93]:
layer3 = PathLayer.from_geopandas(
    path_segments,
    # get_color=[255, 255, 255],
    get_color = colors_planet_swarm,
    get_width=6200,
    opacity=.1,
    auto_highlight=True
)

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [94]:
m3 = Map(
    [layer3],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m3

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…